In [33]:
import deepxde as dde
import numpy as np
from deepxde.backend import tf

In [34]:
# Constant Parameters in the model
Dif = 1e-6
c_0 = 10
c_1 = 3
c_2 = 0
l = 0.5
T_tot = 1.5*l**2/Dif

In [35]:
geom = dde.geometry.Interval(0, l)
timedomain = dde.geometry.TimeDomain(0, T_tot)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

In [36]:
def pde(x, y):
    dy_t = dde.grad.jacobian(y, x, j=1)
    dy_xx = dde.grad.hessian(y, x, j=0)
    return (dy_t - Dif * dy_xx)

In [43]:
def func(x):
    return c_0

def func_bound1(x):
   return c_1

def func_bound2(x):
   return c_2

def boundary1(x,on_boundary):
   return np.isclose(x[0], 0) and on_boundary

def boundary2(x,on_boundary):
   return np.isclose(x[0], l) and on_boundary

In [44]:
bc1 = dde.DirichletBC(geomtime, func_bound1, boundary1)
bc2 = dde.DirichletBC(geomtime, func_bound2, boundary2)
ic = dde.icbc.IC(geomtime, func, lambda _, on_initial: on_initial)

In [45]:
data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc1, bc2, ic],
    num_domain=40,
    num_boundary=20,
    num_initial=10,
    solution=func,
    num_test=10000,
)

In [46]:
layer_size = [2] + [32] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

In [47]:
model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=10000)

Compiling model...
'compile' took 0.009839 s

Training model...

Step      Train loss                                  Test loss                                   Test metric   
0         [1.37e-03, 7.47e+00, 7.16e-02, 9.82e+01]    [2.38e-09, 7.47e+00, 7.16e-02, 9.82e+01]    [9.75e+01]    
1000      [7.51e-07, 2.25e+00, 2.25e+00, 2.24e-02]    [2.40e-07, 2.25e+00, 2.25e+00, 2.24e-02]    [8.51e+01]    
2000      [4.41e-08, 2.25e+00, 2.25e+00, 5.97e-03]    [2.98e-07, 2.25e+00, 2.25e+00, 5.97e-03]    [8.51e+01]    
3000      [8.70e-09, 2.25e+00, 2.25e+00, 2.31e-03]    [3.40e-07, 2.25e+00, 2.25e+00, 2.31e-03]    [8.51e+01]    
4000      [5.89e-08, 2.47e+00, 2.04e+00, 1.09e-03]    [3.46e-07, 2.47e+00, 2.04e+00, 1.09e-03]    [8.58e+01]    
5000      [1.52e-07, 2.25e+00, 2.25e+00, 5.81e-04]    [3.09e-07, 2.25e+00, 2.25e+00, 5.81e-04]    [8.51e+01]    
6000      [3.03e-07, 2.25e+00, 2.25e+00, 3.27e-04]    [2.70e-07, 2.25e+00, 2.25e+00, 3.27e-04]    [8.51e+01]    
7000      [5.39e-07, 2.25e+00, 

In [48]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Saving loss history to /home/gopal2751765/pinns_work/PINNS/loss.dat ...
Saving training data to /home/gopal2751765/pinns_work/PINNS/train.dat ...


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)